# get data body in spark

In [1]:
data = sc.wholeTextFiles('data/reut2-000*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                   .map(lambda x:x.replace('\n', ' '))

newsArticles.take(3)

['Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review.     The dry period means the temporao will be late this year.     Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures.     Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors.     There are doubts as to how much of this cocoa would be fit for expor

# (1) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [2]:
k = 3
shingles = newsArticles.flatMap(lambda x:[x[i:i+k] for i in range(len(x)-k+1)]).distinct()
newsArticles = newsArticles.collect()
kShinglesMatrix = shingles.map(lambda s:[1 if s in a else 0 for a in newsArticles]).collect()

In [3]:
shingles_count = len(kShinglesMatrix)
articles_count = len(kShinglesMatrix[0])
print(shingles_count, 'different shingles.')
print(articles_count, 'different articles.')

14489 different shingles.
925 different articles.


In [4]:
with open('result/kShinglesMatrix.txt', 'w') as result:
    for row in kShinglesMatrix:
        result.write(str(row))

# (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [5]:
h = 10

import random

minHashSignatures = list()
for i in range(h):
    rndIdx = random.sample(range(shingles_count), shingles_count)
    minHash = [-1]*articles_count
    while -1 in minHash:
        row = rndIdx.pop()
        for j in range(len(minHash)):
            if (minHash[j] == -1 or minHash[j] > row) and kShinglesMatrix[row][j] == 1:
                minHash[j] = row
    minHashSignatures.append(minHash)

In [6]:
with open('result/minHashSignatures.txt', 'w') as result:
    for row in minHashSignatures:
        result.write(str(row))